# 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" 


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/claims/claims.csv


In [2]:
# Install required packages for Excel output
!pip install -q openpyxl

# -*- coding: utf-8 -*-
"""BERT.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Q0YHA6RUjwOClr1AKzK3nUcPlVusNKGa
"""


'BERT.ipynb\n\nAutomatically generated by Colab.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1Q0YHA6RUjwOClr1AKzK3nUcPlVusNKGa\n'

In [3]:
# Check if GPU is Available
import torch

if torch.cuda.is_available():
    print("GPU is available.")
    device = torch.device("cuda")
else:
    print("GPU is NOT available.")
    print("Please go to Runtime > Change runtime type > Select GPU")
    device = torch.device("cpu")

GPU is available.


In [4]:
!pip install -q transformers datasets accelerate evaluate optuna

# ===== Imports ===== #
# Import all the libraries

import os
import random
import numpy as np
import pandas as pd
import torch
import json
import openpyxl
import sys
from pathlib import Path
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from evaluate import load as load_metric
from time import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score

E0000 00:00:1745088587.069512    3703 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745088587.077101    3703 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# ===== Seed Everything (for full reproducibility) ===== #
import random
import numpy as np
import torch
import torch.backends.cudnn as cudnn

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False  # This makes sure cuDNN doesn’t auto-tune

set_seed(42)


In [6]:

# ===== Configurations ===== #

# model_name = "bert-base-uncased"  # Use the standard model
# model_name = "distilbert-base-uncased"  
# model_name = "roberta-base"  
# model_name = "albert-base-v2" 
# model_name = "xlnet-base-cased" 
model_name = "google/electra-base-discriminator" 

# lr = 2e-5  # Learning rate (changeable)


timestamp = datetime.now().isoformat()

# Generate a unique run ID with model name and timestamp
# run_id = f"{model_name.replace('/', '_')}_lr{str(lr).replace('.', '')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"


run_id = f"{model_name.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# ===== Environment-Aware Path Setup ===== #
if "google.colab" in sys.modules:
    # Running in Google Colab
    project_root = Path("/content/drive/MyDrive/MSc_Claim_Experiment")
elif Path("/kaggle/working").exists():
    # Running in Kaggle
    project_root = Path("/kaggle/working/MSc_Claim_Experiment")
else:
    # Fallback for local development
    project_root = Path("./MSc_Claim_Experiment")

# Define the output directory for the current run
# output_dir = project_root / "outputs" / "results" / run_id
output_dir = project_root / "outputs" / "results" / model_name.replace("/", "_") / run_id
os.makedirs(output_dir, exist_ok=True)  # Create directory if it does not exist

In [7]:
# ===== Load and Preprocess Data ===== #
df = pd.read_csv("/kaggle/input/claims/claims.csv", encoding="ISO-8859-1", engine="python")
df['rating'] = df['rating'].str.lower().str.strip()  # Make ratings lowercase and remove spaces
mapping = {
    'true': 1,
    'mostly-true': 1,
    'false': 0,
    'mostly-false': 0
}  # Convert ratings to 0 or 1


# Apply label mapping

df['label_binary'] = df['rating'].map(mapping)
df = df[df['label_binary'].isin([0, 1])] # Keep only rows that are 0 or 1 (drop unknowns)

# Split into train, validation, test
train_val, test = train_test_split(df, test_size=0.15, stratify=df['label_binary'], random_state=42)
train, val      = train_test_split(train_val, test_size=0.1765, stratify=train_val['label_binary'], random_state=42)

In [8]:
# ===== Compute Class Weights =====
# Handle class imbalance by adjusting loss contribution of each class. Give rare class more weight
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train['label_binary']),
    y=train['label_binary']
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda' if torch.cuda.is_available() else 'cpu')

# Save class weights to JSON file for reproducibility
with open(output_dir / "class_weights.json", "w") as f:
    json.dump(class_weights.cpu().numpy().tolist(), f, indent=2)

# Show class distribution
print("Train:", train['label_binary'].value_counts(normalize=True))
print("Val:", val['label_binary'].value_counts(normalize=True))
print("Test:", test['label_binary'].value_counts(normalize=True))


Train: label_binary
0.0    0.759352
1.0    0.240648
Name: proportion, dtype: float64
Val: label_binary
0.0    0.761628
1.0    0.238372
Name: proportion, dtype: float64
Test: label_binary
0.0    0.761628
1.0    0.238372
Name: proportion, dtype: float64


In [9]:

# ===== Tokenizer ===== #
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Load BERT tokenizer

def tokenize(batch):
    return tokenizer(batch['statement'], padding=True, truncation=True)  # Turn text into tokens

# Combine all data for consistent indexing
dataset = Dataset.from_pandas(pd.concat([train, val, test]).reset_index(drop=True))
tokenized_dataset = dataset.map(tokenize, batched=True)


# Split tokenized data back into train, val, test
train_dataset = tokenized_dataset.select(range(len(train)))
val_dataset = tokenized_dataset.select(range(len(train), len(train)+len(val)))
test_dataset = tokenized_dataset.select(range(len(train)+len(val), len(dataset)))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1146 [00:00<?, ? examples/s]

In [10]:

# ===== Ensure Labels Are Integers (Fix for loss mismatch) ===== #
train["label_binary"] = train["label_binary"].astype(int)
val["label_binary"] = val["label_binary"].astype(int)
test["label_binary"] = test["label_binary"].astype(int)

In [11]:
# ===== Columns to Remove During Cleanup ===== #
columns_to_remove = [
    'statement', 'label_binary', 'id', 'name', 'description', 'category',
    'rating', 'queries', 'created_at', 'updated_at',
    'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'
]


In [12]:
# Remove unnecessary columns dynamically
def safe_remove_columns(dataset, columns):
    existing = [col for col in columns if col in dataset.column_names]
    return dataset.remove_columns(existing)

In [13]:
# ===== Unified Tokenize + Clean Function ===== #
def tokenize_and_clean(df):
    ds = Dataset.from_pandas(df, preserve_index=False) # Convert to HuggingFace Dataset format
    ds = ds.rename_column("label_binary", "labels") # Rename label column for compatibility
    ds = ds.map(tokenize, batched=True)  # Apply tokenizer to every row
    existing = [col for col in columns_to_remove if col in ds.column_names] # Check which columns can be removed
    return ds.remove_columns(existing) # Return cleaned dataset

In [14]:
# ===== Apply Processing Pipeline to Each Split ===== #
train_ds = tokenize_and_clean(train)
val_ds   = tokenize_and_clean(val)
test_ds  = tokenize_and_clean(test)

print("Final dataset columns:", train_ds.column_names) # Only 'input_ids', 'attention_mask', 'labels'

Map:   0%|          | 0/802 [00:00<?, ? examples/s]

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

Final dataset columns: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']


In [15]:
# ===== Initialize Model ===== #
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def compute_metrics(pred):

    y_true = pred.label_ids # The true labels of the test set
    y_pred = pred.predictions.argmax(-1) # The model's predicted class (the one with the highest probability)

    # Calculate evaluation metrics for class 0
    precision = precision_score(y_true, y_pred, pos_label=0, zero_division=0)
    recall = recall_score(y_true, y_pred, pos_label=0, zero_division=0)
    f1 = f1_score(y_true, y_pred, pos_label=0, zero_division=0)

    # Overall accuracy of the model
    acc = accuracy_score(y_true, y_pred)

    # Calculate weighted average F1 score (accounts for class imbalance)
    f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    # Calculate micro-averaged precision, recall, and F1 (global average across all classes)
    precision_micro = precision_score(y_true, y_pred, average="micro", zero_division=0)
    recall_micro = recall_score(y_true, y_pred, average="micro", zero_division=0)
    f1_micro = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # Return all metrics in a dictionary
    return {
        'eval_f1': f1_weighted,
        "precision": precision,
        "recall": recall,
        # "f1": f1,
        "f1_weighted": f1_weighted,
        "accuracy": acc,
        "precision_micro": precision_micro,
        "recall_micro": recall_micro,
        "precision_weighted": precision_score(y_true, y_pred, average="weighted"),
        "recall_weighted": recall_score(y_true, y_pred, average="weighted"),
        "f1_micro": f1_micro,
        "support_1": int((y_true == 1).sum()),  # Number of class 1 samples
        "support_0": int((y_true == 0).sum())  # Number of class 0 samples
    }

In [17]:
# ===== Define Training Arguments ===== #
# Key parameters for reproducibility and fine-tuning control
# This block defines how training should proceed, how models are saved, and evaluation behavior

training_args = TrainingArguments(
    output_dir=output_dir,  # Folder to store model checkpoints and logs
    # evaluation_strategy='epoch',  # Evaluate model after each epoch
    eval_strategy='epoch', 
    save_strategy='epoch', # Save model after each epoch
    save_total_limit=1, # Keep only the latest saved model to reduce disk usage
    # learning_rate=2e-5,  # Initial learning rate for optimizer
    # learning_rate=3e-5, # Alternative setting for grid search
    learning_rate=5e-5, # # Max learning rate
    lr_scheduler_type="cosine",
    warmup_ratio=0.1, 
    per_device_train_batch_size=8,  # Number of samples per GPU/CPU during training
    per_device_eval_batch_size=8,  # Number of samples per GPU/CPU during validation
    num_train_epochs=100, # Max number of training epochs (early stopping can interrupt before 100)
    weight_decay=0.01, # L2 regularization to reduce overfitting
    load_best_model_at_end=True,  # Restore best model at the end of training based on eval metric
    metric_for_best_model = 'eval_f1',
    greater_is_better = True,
    logging_dir=f"logs/{run_id}", # Directory to save training logs
    logging_strategy='epoch', # Log metrics at the end of each epoch
    seed=42, # Set random seed for reproducibility
    fp16=True,  # Use mixed precision for faster training on supported GPUs
    remove_unused_columns=False,  # Retain all columns to prevent loss of important features
    report_to='none', # Do not report metrics to third-party tools (e.g., WandB)
    disable_tqdm=True,  # Disable tqdm progress bar to reduce console clutter in Colab
    eval_accumulation_steps=32,  # Accumulate gradients during evaluation to save memory
    no_cuda=False  # Use GPU if available
)

In [18]:
# ===== Custom Logging Callback for Per-Epoch Metrics ===== #
# This callback logs evaluation metrics (loss, accuracy, precision, recall, F1) after every epoch
# It writes the results into a CSV file "epoch_log.csv" for visualization or debugging

from transformers import TrainerCallback
import csv

class MetricsLoggerCallback(TrainerCallback):
    def __init__(self, log_file=output_dir / "epoch_log.csv"):
        self.log_file = log_file
        with open(self.log_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                "epoch", "learning_rate",
                "eval_loss", "accuracy", "precision", "recall", "f1"
            ])

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None or "eval_loss" not in logs:
            return  # Skip non-eval logs

        current_lr = logs.get("learning_rate", None)

        with open(self.log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                round(state.epoch, 2),
                current_lr,
                logs.get("eval_loss"),
                logs.get("eval_accuracy"),
                logs.get("eval_precision"),
                logs.get("eval_recall"),
                logs.get("eval_f1")
            ])

In [1]:

# ===== Custom Trainer with Class Weights =====
#  Enables weighted loss to address label imbalance in binary classification
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Use weighted cross entropy loss
        #loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        #loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        
        num_labels = model.module.config.num_labels if hasattr(model, "module") else model.config.num_labels

        loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))

        # loss = loss_fct(logits.view(-1, model.module.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss


# Instantiate Trainer with early stopping and epoch logger
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5), MetricsLoggerCallback()]
)

2025-04-19 19:58:53.182645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745092733.484710      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745092733.566766      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


NameError: name 'model' is not defined

In [20]:
# ===== Train Model with Timer ===== #
# Record how long training took
start_time = time()
trainer.train()
end_time = time()
train_time = round(end_time - start_time, 2)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.6776, 'grad_norm': 248065.46875, 'learning_rate': 4.901960784313726e-06, 'epoch': 1.0}
{'eval_f1': 0.550808975727147, 'eval_loss': 0.6644456386566162, 'eval_precision': 0.9166666666666666, 'eval_recall': 0.4198473282442748, 'eval_f1_weighted': 0.550808975727147, 'eval_accuracy': 0.5290697674418605, 'eval_precision_micro': 0.5290697674418605, 'eval_recall_micro': 0.5290697674418605, 'eval_precision_weighted': 0.7747785160575859, 'eval_recall_weighted': 0.5290697674418605, 'eval_f1_micro': 0.5290697674418605, 'eval_support_1': 41, 'eval_support_0': 131, 'eval_runtime': 2.2146, 'eval_samples_per_second': 77.666, 'eval_steps_per_second': 4.967, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.6131, 'grad_norm': 462267.03125, 'learning_rate': 9.901960784313725e-06, 'epoch': 2.0}
{'eval_f1': 0.6483583282848889, 'eval_loss': 0.5880460143089294, 'eval_precision': 0.9230769230769231, 'eval_recall': 0.549618320610687, 'eval_f1_weighted': 0.6483583282848889, 'eval_accuracy': 0.622093023255814, 'eval_precision_micro': 0.622093023255814, 'eval_recall_micro': 0.622093023255814, 'eval_precision_weighted': 0.7917967114528223, 'eval_recall_weighted': 0.622093023255814, 'eval_f1_micro': 0.622093023255814, 'eval_support_1': 41, 'eval_support_0': 131, 'eval_runtime': 2.1078, 'eval_samples_per_second': 81.603, 'eval_steps_per_second': 5.219, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


...

{'loss': 0.143, 'grad_norm': 3.67734694480896, 'learning_rate': 3.990196078431373e-05, 'epoch': 8.0}
{'eval_f1': 0.7967354171614388, 'eval_loss': 3.566232681274414, 'eval_precision': 0.8540145985401459, 'eval_recall': 0.8931297709923665, 'eval_f1_weighted': 0.7967354171614388, 'eval_accuracy': 0.8023255813953488, 'eval_precision_micro': 0.8023255813953488, 'eval_recall_micro': 0.8023255813953488, 'eval_precision_weighted': 0.7934646070276693, 'eval_recall_weighted': 0.8023255813953488, 'eval_f1_micro': 0.8023255813953488, 'eval_support_1': 41, 'eval_support_0': 131, 'eval_runtime': 2.3478, 'eval_samples_per_second': 73.261, 'eval_steps_per_second': 4.685, 'epoch': 8.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.165, 'grad_norm': 0.014997675083577633, 'learning_rate': 4.490196078431373e-05, 'epoch': 9.0}
{'eval_f1': 0.7186175392370261, 'eval_loss': 4.709323406219482, 'eval_precision': 0.8910891089108911, 'eval_recall': 0.6870229007633588, 'eval_f1_weighted': 0.7186175392370261, 'eval_accuracy': 0.6976744186046512, 'eval_precision_micro': 0.6976744186046512, 'eval_recall_micro': 0.6976744186046512, 'eval_precision_weighted': 0.7793989356354568, 'eval_recall_weighted': 0.6976744186046512, 'eval_f1_micro': 0.6976744186046512, 'eval_support_1': 41, 'eval_support_0': 131, 'eval_runtime': 2.1455, 'eval_samples_per_second': 80.167, 'eval_steps_per_second': 5.127, 'epoch': 9.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.2859, 'grad_norm': 0.06002817302942276, 'learning_rate': 4.990196078431373e-05, 'epoch': 10.0}
{'eval_f1': 0.7556185921480371, 'eval_loss': 4.158874988555908, 'eval_precision': 0.8717948717948718, 'eval_recall': 0.7786259541984732, 'eval_f1_weighted': 0.7556185921480371, 'eval_accuracy': 0.7441860465116279, 'eval_precision_micro': 0.7441860465116279, 'eval_recall_micro': 0.7441860465116279, 'eval_precision_weighted': 0.7766682929473627, 'eval_recall_weighted': 0.7441860465116279, 'eval_f1_micro': 0.7441860465116278, 'eval_support_1': 41, 'eval_support_0': 131, 'eval_runtime': 2.2151, 'eval_samples_per_second': 77.648, 'eval_steps_per_second': 4.966, 'epoch': 10.0}
{'train_runtime': 296.633, 'train_samples_per_second': 270.368, 'train_steps_per_second': 17.193, 'train_loss': 0.31720276065901215, 'epoch': 10.0}


In [21]:
#  Save training loss history (for reproducibility / visualization)
with open(output_dir / "train_loss_curve.json", "w") as f:
    json.dump(trainer.state.log_history, f, indent=2)

# ===== Save training time ===== #
with open(output_dir / "training_time_log.json", "w") as f:
    json.dump({"train_time_seconds": train_time}, f, indent=2)


In [22]:
print(f"Model loaded from: {trainer.model.base_model_prefix}, current LR: {trainer.optimizer.param_groups[0]['lr']}")

Model loaded from: electra, current LR: 5e-05


In [23]:
from transformers import AutoModelForSequenceClassification


# ===== Predict and Save Outputs ===== #
# Generate predictions from best model on the test set
preds = trainer.predict(test_ds)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [24]:
# Save the best model weights for reproducibility or future use
trainer.save_model(project_root / "outputs" / "results" / "saved_models" / f"{model_name.replace('/', '_')}_{run_id}")

In [25]:
# Extract predicted labels
preds_labels = np.argmax(preds.predictions, axis=-1)

In [26]:
# ===== Save Raw Prediction Table ===== #
# Save full test set results with input, true label, predicted label, and logits
results = pd.DataFrame({
    'statement': test['statement'].reset_index(drop=True),
    'true_label': test['label_binary'].reset_index(drop=True),
    'pred_label': preds_labels,
    'logits': preds.predictions.tolist(),
    'model_name': model_name,
    'run_id': run_id
})
results.to_csv(output_dir / "prediction.csv", index=False)

In [27]:
# ===== Save Metrics =====
# Save raw evaluation metrics to JSON
with open(output_dir / "eval_metrics.json", "w") as f:
    json.dump(preds.metrics, f, indent=2)

In [28]:
# ===== Save Report & Confusion Matrix =====
# Save detailed classification report and confusion matrix
conf_matrix = confusion_matrix(results['true_label'], results['pred_label'])
report = classification_report(results['true_label'], results['pred_label'], output_dict=True)

with open(output_dir / "classification_report.json", "w") as f:
    json.dump(report, f, indent=2)

with open(output_dir / "confusion_matrix.json", "w") as f:
    json.dump(conf_matrix.tolist(), f, indent=2)


In [29]:
# === Estimate learning rate & early stop epoch ===
best_lr = trainer.optimizer.param_groups[0]['lr']
steps_per_epoch = len(train_ds) // training_args.per_device_train_batch_size
epoch_stopped_at = trainer.state.global_step // steps_per_epoch

print(f"Final learning rate (from best checkpoint): {best_lr}")
print(f"Early stopped at epoch: {epoch_stopped_at}")

# save best_lr to file
with open(output_dir / "best_lr.txt", "w") as f:
    f.write(str(best_lr))

Final learning rate (from best checkpoint): 5e-05
Early stopped at epoch: 5


In [30]:
# Save model & training configuration before training
config_snapshot = {
    "model_name": model_name,
    "run_id": run_id,

     # Hyperparameters
    "learning_rate": training_args.learning_rate,
    "lr_scheduler_type": training_args.lr_scheduler_type,
    "warmup_ratio": training_args.warmup_ratio,
    "batch_size": training_args.per_device_train_batch_size,
    "num_epochs": training_args.num_train_epochs,
    "weight_decay": training_args.weight_decay,
    "early_stopping": True,

    "best_lr": best_lr,
    "epoch_stopped_at": epoch_stopped_at,
    
    "class_weights": class_weights.cpu().tolist(),
     "datetime": timestamp
    #"datetime": datetime.now().isoformat()
}
with open(output_dir / "config.json", "w") as f:
    json.dump(config_snapshot, f, indent=2)

# Write human-readable run info for easy reference
with open(output_dir / "README.txt", "w") as f:
    f.write(f"Run ID: {run_id}\nModel: {model_name}\nCreated: {timestamp}\n")
    f.write(f"Best LR (from checkpoint): {best_lr}\nStopped at epoch: {epoch_stopped_at}\n")

In [31]:
# ===== Save TrainingArguments Params (for reproducibility) ===== #
with open(output_dir / "training_args.json", "w") as f:
    json.dump(training_args.to_dict(), f, indent=2)

# ===== Print summary ===== #
print(f"All results saved to: {output_dir}")



# ===== Append Main Metrics to Excel Summary Table ===== #
# Create/update a summary Excel for final model comparisons
summary_path = project_root / "outputs" / "results" / "final_model_comparison.xlsx"

# Re-run prediction to ensure use final model output
preds = trainer.predict(test_ds)
y_true = preds.label_ids
y_pred = preds.predictions.argmax(-1)

# Compute detailed classification report
report = classification_report(
    y_true, y_pred,
    output_dict=True,
    zero_division=0
)

# Manually compute micro metrics to avoid missing values
precision_micro = precision_score(y_true, y_pred, average='micro', zero_division=0)
recall_micro = recall_score(y_true, y_pred, average='micro', zero_division=0)
f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0)

# One row summarizing performance of this model run
summary_row = {
    "model_name": model_name,
    "run_id": run_id,
    "accuracy": preds.metrics.get("test_accuracy", preds.metrics.get("eval_accuracy", report["accuracy"])),

    # Per-class metrics
    "precision_0": report['0']['precision'],
    "recall_0": report['0']['recall'],
    "f1_0": report['0']['f1-score'],
    "precision_1": report['1']['precision'],
    "recall_1": report['1']['recall'],
    "f1_1": report['1']['f1-score'],

    # Micro-average
    "precision_micro": precision_micro,
    "recall_micro": recall_micro,
    "f1_micro": f1_micro,

    # Averages
    "f1_macro": report['macro avg']['f1-score'],
    "f1_weighted": report['weighted avg']['f1-score'],
    "precision_weighted": report['weighted avg']['precision'], 
    "recall_weighted": report['weighted avg']['recall'], 

    "lr_max": 5e-5,
    "lr_min": 1e-6,  
    "best_lr": best_lr,
    "epoch_stopped_at": epoch_stopped_at,

     # Class balance info
    "support_1": report['1']['support'],
    "support_0": report['0']['support'],
    "train_time_sec": train_time

}

# ===== Excel Integration ===== #
# Append or create Excel summary table for all runs

df_row = pd.DataFrame([summary_row])
if not summary_path.exists():
    df_row.to_excel(summary_path, index=False)
else:
    existing = pd.read_excel(summary_path)
    updated = pd.concat([existing, df_row], ignore_index=True)
    updated.to_excel(summary_path, index=False)

print(f"Summary row saved to {summary_path}")

All results saved to: /kaggle/working/MSc_Claim_Experiment/outputs/results/google_electra-base-discriminator/google_electra-base-discriminator_20250419_184950


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Summary row saved to /kaggle/working/MSc_Claim_Experiment/outputs/results/final_model_comparison.xlsx


In [32]:
import os

base_path = "/kaggle/working/MSc_Claim_Experiment/outputs/results"

for root, dirs, files in os.walk(base_path):
    for file in files:
        print(os.path.join(root, file))

/kaggle/working/MSc_Claim_Experiment/outputs/results/final_model_comparison.xlsx
/kaggle/working/MSc_Claim_Experiment/outputs/results/google_electra-base-discriminator/google_electra-base-discriminator_20250419_184950/README.txt
/kaggle/working/MSc_Claim_Experiment/outputs/results/google_electra-base-discriminator/google_electra-base-discriminator_20250419_184950/confusion_matrix.json
/kaggle/working/MSc_Claim_Experiment/outputs/results/google_electra-base-discriminator/google_electra-base-discriminator_20250419_184950/class_weights.json
/kaggle/working/MSc_Claim_Experiment/outputs/results/google_electra-base-discriminator/google_electra-base-discriminator_20250419_184950/config.json
/kaggle/working/MSc_Claim_Experiment/outputs/results/google_electra-base-discriminator/google_electra-base-discriminator_20250419_184950/best_lr.txt
/kaggle/working/MSc_Claim_Experiment/outputs/results/google_electra-base-discriminator/google_electra-base-discriminator_20250419_184950/classification_report

In [33]:
import pandas as pd
from pathlib import Path

# === 1. Define Base Path Depending on Platform ===
if Path("/kaggle/working").exists():
    base_path = Path("/kaggle/working/MSc_Claim_Experiment/outputs/results")
elif Path("/content/drive").exists():
    base_path = Path("/content/drive/MyDrive/MSc_Claim_Experiment/outputs/results")
else:
    base_path = Path("./outputs/results")  # fallback for local dev

# === 2. Load model summary file ===
df_summary = pd.read_excel(base_path / "final_model_comparison.xlsx")
print("Final Model Comparison:")
display(df_summary.head())

# === 3. Define model_keyword (before calling the function) ===
model_keyword = "None"  # Change to "bert", "distilbert", or None

# === 4. Define reusable function ===
def get_latest_run_folder(results_path: Path, model_keyword: str = None) -> Path:
    """
    Get the most recently updated results folder.
    If model_keyword is provided, only consider folders containing that keyword.
    """
    subdirs = [d for d in results_path.iterdir() if d.is_dir()]
    if model_keyword:
        subdirs = [d for d in subdirs if model_keyword.lower() in d.name.lower()]
    return max(subdirs, key=lambda d: d.stat().st_mtime) if subdirs else None

# === 5. Get latest result folder and load predictions ===
latest_run_folder = get_latest_run_folder(base_path, model_keyword=model_keyword)

if latest_run_folder:
    df_pred = pd.read_csv(latest_run_folder / "prediction.csv")
    print(f"Sample Predictions from: {latest_run_folder.name}")
    display(df_pred.head())
else:
    print("No valid result folder found.")

Final Model Comparison:


,model_name,run_id,accuracy,precision_0,recall_0,f1_0,precision_1,recall_1,f1_1,precision_micro,...,f1_weighted,precision_weighted,recall_weighted,lr_max,lr_min,best_lr,epoch_stopped_at,support_1,support_0,train_time_sec
0,albert-base-v2,albert-base-v2_20250419_182934,0.802326,0.843972,0.908397,0.87500,0.612903,0.463415,0.527778,0.802326,...,0.792232,0.788891,0.802326,0.00005,0.000001,0.000035,3,41,131,93.88
1,xlnet-base-cased,xlnet-base-cased_20250419_183811,0.854651,0.895522,0.916031,0.90566,0.710526,0.658537,0.683544,0.854651,...,0.852714,0.851424,0.854651,0.00005,0.000001,0.000050,5,41,131,358.68
2,google/electra-base-discriminator,google_electra-base-discriminator_20250419_184950,0.802326,0.859259,0.885496,0.87218,0.594595,0.536585,0.564103,0.802326,...,0.798743,0.796171,0.802326,0.00005,0.000001,0.000050,5,41,131,297.34


No valid result folder found.


In [34]:
# import shutil

# Backup all output files (including intermediate files, models, logs, etc.)
# This creates a full archive of the experiment outputs
# shutil.make_archive(
#     "/kaggle/working/results_backup", 
#    'zip', 
#    "/kaggle/working/MSc_Claim_Experiment/outputs"
# )

